<a href="https://colab.research.google.com/github/HanqiLouis/GFET-Characterization/blob/main/Plot_Average_GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
# @title Plot Average Runs
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output


### ------- Plot Functions -------
def plot_average_runs(groups_of_folders, names_of_runs, site_number, runs_per_site_list, dual_sweep_list, save_output=None, display=False):
    """
    Plots the average runs for specific groups of folders on the same plot and optionally saves the plot.

    Parameters:
    - groups_of_folders: list of list of str, each sublist contains paths to folders representing a group.
    - names_of_runs: list of str, names of the groups for labeling.
    - site_number: int, the site to plot (1-indexed).
    - runs_per_site_list: list of int, each value represents the runs per site for the corresponding group.
    - dual_sweep_list: list of bool, each value represents whether dual sweep is enabled for the corresponding group.
    - save_output: str or None, path to the folder where the plot should be saved (default: None).
    """

    # Iterate through each group of folders
    for group_index, (folder_group, runs_per_site, dual_sweep) in enumerate(zip(groups_of_folders, runs_per_site_list, dual_sweep_list)):
        all_runs = []  # To store all runs for averaging in the current group

        # Helper function to get files for a site from a folder
        def get_site_files(directory_path):
            start_index = (site_number - 1) * runs_per_site
            end_index = start_index + runs_per_site
            excel_files = sorted(glob.glob(os.path.join(directory_path, '*.xlsx')))
            return excel_files[start_index:end_index]

        # Iterate through folders in the current group
        for folder_index, directory_path in enumerate(folder_group):
            site_files = get_site_files(directory_path)

            if not site_files:
                print(f"No files found for Site {site_number} in folder {directory_path}.")
                continue

            # Collect runs from the current folder
            for file in site_files:
                df = pd.read_excel(file)

                # Apply dual sweep logic for this specific group
                if dual_sweep:
                    total_rows = df.shape[0]
                    df_subset = df.iloc[0:int(total_rows / 2)]
                    Vg = df_subset['VG']
                    Id = df_subset['ID']
                else:
                    Vg = df['VG']
                    Id = df['ID']

                all_runs.append((1000 * Vg, 1000000 * Id))  # Convert units

        # Compute and plot the average run for the current group
        if all_runs:
            # Interpolate to align all runs to the same VG points
            common_Vg = np.linspace(
                min(run[0].min() for run in all_runs),
                max(run[0].max() for run in all_runs),
                500  # Number of points for interpolation
            )
            interpolated_Id = [np.interp(common_Vg, Vg, Id) for Vg, Id in all_runs]

            avg_Id = np.mean(interpolated_Id, axis=0)

            plt.plot(
                common_Vg,
                avg_Id,
                linestyle='-',
                linewidth=2,
                label=f'{names_of_runs[group_index]}'
            )

    # Add labels, title, and legend
    plt.xlabel('VG [mV]')
    plt.ylabel('ID [uA]')
    plt.title(f'Site {site_number} - Average Runs Comparison')
    plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1))
    plt.tight_layout()
    plt.grid()

    # Save the plot if save_output is specified
    if save_output:
        os.makedirs(save_output, exist_ok=True)  # Create the directory if it doesn't exist
        plot_path = os.path.join(save_output, f'site_{site_number}_average_runs_plot.png')
        plt.savefig(plot_path, dpi=300)
        print(f"Plot saved to {plot_path}")

    if display:
        plt.show()
    else:
        plt.close()



### ------- Global Variables -------
groups = []  # List of groups containing folders
output = widgets.Output()  # Output area for displaying information

### ------- Group Class -------
class Group:
    def __init__(self, index):
        """ Initialize a group with name, runs per site, dual sweep checkbox, and folders """
        self.index = index

        # Group name field
        self.group_name = widgets.Text(
            description=f'Measurement Group {index} Name:',
            placeholder=f'Enter name for Measurement Group {index}',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='750px')
        )

        # Runs per site (positive integer)
        self.runs_per_site = widgets.BoundedIntText(
            value=0, min=0,
            description="Runs per site:",
            style={'description_width': 'initial'}
        )

        # Checkbox for dual sweep
        self.dual_sweep = widgets.Checkbox(
            value=False,
            description="Dual sweep"
        )

        # Folder storage
        self.folders = []
        self.add_folder()  # Ensure at least one folder initially

        # Buttons for adding/removing folders
        self.add_folder_button = widgets.Button(description="+ Folder", button_style="success")
        self.remove_folder_button = widgets.Button(description="- Folder", button_style="danger")

        self.add_folder_button.on_click(self.add_folder)
        self.remove_folder_button.on_click(self.remove_folder)

    def add_folder(self, _=None):
        """ Add a new folder to this group """
        folder_widget = widgets.Text(
            description=f'Folder {len(self.folders) + 1}:',
            placeholder=f'Enter path for Folder {len(self.folders) + 1}',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='750px')
        )
        self.folders.append(folder_widget)
        update_ui()

    def remove_folder(self, _=None):
        """ Remove the last folder, ensuring at least one remains """
        if len(self.folders) > 1:
            self.folders.pop()
            update_ui()

### ------- UI Update Function -------
def update_ui():
    """ Clears and updates the UI with the latest state of widgets """
    clear_output(wait=True)

    # Display all groups and their folders
    for group in groups:
        display(group.group_name)

        for folder in group.folders:
            display(folder)

        display(group.runs_per_site)
        display(group.dual_sweep)
        display(widgets.HBox([group.add_folder_button, group.remove_folder_button]))
        display(widgets.HTML('<hr>'))  # Separator

    # Display global controls
    display(widgets.HBox([add_group_button, remove_group_button]))
    display(nb_sites)
    display(site_to_display)
    display(save_checkbox)
    display(save_directory_field)
    display(plot_button, output)
    display(save_all_button)

### ------- Group Management Functions -------
def add_group(_=None):
    """ Add a new group with an initial folder """
    groups.append(Group(len(groups) + 1))
    update_ui()

def remove_group(_=None):
    """ Remove the last added group, ensuring at least one remains """
    if len(groups) > 1:
        groups.pop()
        update_ui()

### ------- Plot & Save Functions -------
def validate_inputs():
    """ Check if required fields are filled before plotting """
    with output:
        output.clear_output()

        for group in groups:
            if group.runs_per_site.value == 0:
                print(f"{group.group_name.value or f'Measurement {group.index}'}: Please enter the number of runs per site")
                return False

            for idx, folder in enumerate(group.folders, start=1):
                if not folder.value.strip():
                    print(f"{group.group_name.value or f'Measurement {group.index}'}: No input provided for Folder {idx}")
                    return False

        if nb_sites.value == 0:
            print("Please enter the number of sites")
            return False

        if site_to_display.value == 0:
            print("Please enter the site to display")
            return False

        return True

def collect_group_data():
    """ Collect and structure group data for plotting """
    groups_list, groups_names, runs_per_site_list, dual_sweep_list = [], [], [], []

    for group in groups:
        groups_names.append(group.group_name.value or f"Measurement {group.index}")
        runs_per_site_list.append(group.runs_per_site.value)
        dual_sweep_list.append(group.dual_sweep.value)
        groups_list.append([folder.value.strip() for folder in group.folders])

    return groups_list, groups_names, runs_per_site_list, dual_sweep_list

def plot_average(_=None):
    """ Process inputs and call plot function """
    with output:
        output.clear_output()

        if not validate_inputs():
            return

        groups_list, groups_names, runs_per_site_list, dual_sweep_list = collect_group_data()

        if save_checkbox.value and not save_directory_field.value.strip():
            print("Please enter the save directory")
            return

        plot_average_runs(groups_list, groups_names, site_to_display.value, runs_per_site_list, dual_sweep_list,
                          save_output=save_directory_field.value.strip() if save_checkbox.value else None, display=True)

def save_all_average_plots(_=None):
    """ Save all plots for each site """
    if not validate_inputs():
        return

    if not save_directory_field.value.strip():
        print("Please enter the save directory")
        return

    groups_list, groups_names, runs_per_site_list, dual_sweep_list = collect_group_data()

    for i in range(nb_sites.value):
        plot_average_runs(groups_list, groups_names, i + 1, runs_per_site_list, dual_sweep_list,
                          save_output=save_directory_field.value.strip(), display=False)

### ------- Widgets -------
nb_sites = widgets.BoundedIntText(value=0, min=0, description="Number of sites:", style={'description_width': 'initial'})
site_to_display = widgets.BoundedIntText(value=0, min=0, description="Site to display:", style={'description_width': 'initial'})

save_checkbox = widgets.Checkbox(value=False, description='Save', style={'description_width': 'initial'})
save_directory_field = widgets.Text(
    description='Save Directory:',
    placeholder='Enter directory to save files',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='750px')
)

add_group_button = widgets.Button(description="+ Group", button_style="success")
remove_group_button = widgets.Button(description="- Group", button_style="danger")
add_group_button.on_click(add_group)
remove_group_button.on_click(remove_group)

plot_button = widgets.Button(description='Plot', button_style='warning')
plot_button.on_click(plot_average)

save_all_button = widgets.Button(description='Save all plots', button_style='primary')
save_all_button.on_click(save_all_average_plots)

### ------- Initialize UI -------
add_group(None)
update_ui()


Text(value='', description='Measurement Group 1 Name:', layout=Layout(width='750px'), placeholder='Enter name …

Text(value='', description='Folder 1:', layout=Layout(width='750px'), placeholder='Enter path for Folder 1', s…

Text(value='', description='Folder 2:', layout=Layout(width='750px'), placeholder='Enter path for Folder 2', s…

BoundedIntText(value=0, description='Runs per site:', style=DescriptionStyle(description_width='initial'))

Checkbox(value=False, description='Dual sweep')

...

Checkbox(value=False, description='Save', style=DescriptionStyle(description_width='initial'))

Text(value='', description='Save Directory:', layout=Layout(width='750px'), placeholder='Enter directory to sa…

Button(button_style='warning', description='Plot', style=ButtonStyle())

Output()

Button(button_style='primary', description='Save all plots', style=ButtonStyle())